In [1]:
import random
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from pricer.evaluator import evaluate
from pricer.items import Item

In [2]:
LITE_MODE = True

In [3]:
username = "ed-donner"
dataset = f"{username}/items_lite"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


In [4]:
def random_pricer(item):
    return random.randrange(1,1000)

In [5]:
random.seed(42)
evaluate(random_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$436 $1 $29 $690 $252 $21 $85 $72 $719 $225 $20 $380 $894 $505 $11 $572 $354 $17 $179 $23 $90 $115 $433 $442 $304 $122 $291 $714 $567 $639 $539 $370 $66 $380 $489 $534 $769 $835 $207 $740 $626 $84 $680 $178 $129 $260 $142 $189 $836 $580 $310 $25 $380 $270 $47 $234 $861 $313 $417 $259 $591 $33 $657 $361 $79 $38 $757 $500 $263 $5 $534 $284 $570 $625 $584 $871 $759 $361 $575 $178 $602 $60 $17 $579 $207 $732 $115 $224 $756 $193 $866 $9 $370 $250 $456 $423 $821 $217 $103 $195 $264 $98 $650 $135 $470 $842 $675 $264 $43 $325 $591 $138 $516 $619 $56 $2 $449 $369 $221 $845 $640 $616 $501 $59 $502 $273 $844 $688 $616 $81 $164 $705 $52 $795 $259 $396 $70 $2 $89 $798 $902 $331 $818 $716 $129 $186 $627 $2 $141 $873 $918 $553 $423 $7 $253 $136 $204 $707 $255 $502 $19 $739 $557 $426 $80 $575 $39 $321 $185 $132 $497 $484 $326 $751 $53 $733 $85 $64 $549 $71 $158 $672 $133 $362 $16 $373 $258 $544 $420 $515 $223 $944 $532 $743 $866 $68 $527 $459 $67 $673 

In [6]:


training_prices = [item.price for item in train]
training_average = sum(training_prices) / len(training_prices)
print(training_average)

def constant_pricer(item):
    return training_average

140.347293


In [7]:
evaluate(constant_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$79 $24 $85 $70 $110 $90 $4 $75 $105 $190 $573 $239 $120 $86 $61 $107 $61 $90 $70 $21 $6 $16 $55 $35 $192 $312 $355 $120 $42 $60 $120 $80 $20 $60 $25 $679 $81 $84 $73 $102 $59 $60 $105 $115 $80 $115 $122 $109 $5 $62 $105 $10 $335 $21 $87 $6 $133 $100 $62 $128 $96 $62 $49 $30 $488 $50 $100 $305 $15 $64 $109 $123 $140 $121 $90 $104 $16 $130 $123 $122 $20 $128 $110 $41 $113 $80 $42 $166 $20 $95 $118 $45 $120 $105 $132 $88 $106 $17 $130 $435 $40 $23 $103 $1 $109 $22 $115 $260 $109 $159 $80 $174 $109 $12 $55 $30 $116 $120 $84 $37 $124 $51 $70 $26 $60 $80 $120 $41 $39 $1 $69 $3 $55 $110 $75 $125 $65 $70 $12 $2 $76 $110 $60 $121 $54 $108 $95 $370 $125 $107 $121 $34 $93 $0 $121 $139 $91 $84 $179 $90 $149 $114 $98 $127 $700 $117 $307 $90 $100 $130 $115 $118 $280 $118 $39 $9 $112 $47 $46 $47 $514 $115 $160 $108 $90 $118 $7 $73 $80 $114 $105 $89 $105 $2 $40 $60 $30 $140 $89 $114 

In [8]:
def get_features(item):
    return {
        "weight": item.weight,
        "weight_unknown": 1 if item.weight==0 else 0,
        "text_length": len(item.summary)
    }

In [9]:
def list_to_dataframe(items):
    features = [get_features(item) for item in items]
    df = pd.DataFrame(features)
    df['price'] = [item.price for item in items]
    return df

train_df = list_to_dataframe(train)
test_df = list_to_dataframe(test)

In [12]:
# Traditional Linear Regression!

np.random.seed(42)

# Separate features and target
feature_columns = ['weight', 'weight_unknown', 'text_length']

X_train = train_df[feature_columns]
y_train = train_df['price']
X_test = test_df[feature_columns]
y_test = test_df['price']

# Train a Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

for feature, coef in zip(feature_columns, model.coef_):
    print(f"{feature}: {coef}")
print(f"Intercept: {model.intercept_}")

# Predict the test set and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

weight: 3.568744414255301
weight_unknown: 20.90175988023751
text_length: 0.20343123739152272
Intercept: 40.912387807913376
Mean Squared Error: 20096.925335647466
R-squared Score: 0.1609102308229894


In [13]:
def linear_regression_pricer(item):
    features = get_features(item)
    features_df = pd.DataFrame([features])
    return model.predict(features_df)[0]

In [14]:
evaluate(linear_regression_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$79 $51 $62 $66 $71 $50 $9 $50 $82 $191 $574 $233 $113 $58 $28 $97 $50 $62 $39 $14 $18 $14 $75 $11 $193 $318 $359 $91 $35 $36 $104 $78 $27 $16 $79 $676 $65 $66 $70 $67 $64 $36 $61 $66 $104 $93 $104 $83 $25 $60 $84 $5 $347 $2 $79 $20 $110 $83 $23 $123 $111 $49 $19 $3 $241 $17 $82 $261 $6 $68 $85 $105 $155 $89 $91 $80 $3 $98 $99 $88 $72 $86 $86 $8 $84 $43 $81 $176 $62 $60 $88 $42 $80 $71 $91 $117 $78 $8 $146 $418 $33 $7 $106 $21 $122 $1 $83 $298 $104 $198 $62 $154 $106 $59 $71 $53 $103 $94 $82 $14 $99 $43 $51 $51 $127 $47 $86 $102 $72 $16 $53 $9 $47 $72 $45 $84 $81 $40 $9 $11 $39 $132 $55 $85 $3 $71 $71 $364 $5 $72 $93 $26 $53 $21 $90 $119 $103 $40 $43 $79 $160 $92 $67 $116 $725 $100 $279 $74 $84 $101 $86 $100 $244 $78 $139 $58 $95 $7 $50 $38 $289 $117 $38 $94 $75 $96 $6 $77 $57 $79 $79 $78 $108 $21 $14 $97 $50 $109 $90 $114 

In [15]:
prices = np.array([float(item.price) for item in train])
documents = [item.summary for item in train]

In [16]:
np.random.seed(42)
vectorizer = CountVectorizer(max_features=2000, stop_words='english')
X = vectorizer.fit_transform(documents)

In [17]:
# Here are the 1,000 most common words that it picked, not including "stop words":

selected_words = vectorizer.get_feature_names_out()
print(f"Number of selected words: {len(selected_words)}")
print("Selected words:", selected_words[1000:1020])

Number of selected words: 2000
Selected words: ['jacket' 'jeep' 'jigsaw' 'joint' 'joints' 'keeping' 'keeps' 'key'
 'keyboard' 'keypad' 'keys' 'kg' 'khz' 'kia' 'kickstand' 'kids' 'king'
 'kingston' 'kit' 'kitchen']


In [18]:
regressor = LinearRegression()
regressor.fit(X, prices)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [19]:
def natural_language_linear_regression_pricer(item):
    x = vectorizer.transform([item.summary])
    return max(regressor.predict(x)[0], 0)

In [20]:
evaluate(natural_language_linear_regression_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$87 $122 $55 $70 $191 $229 $22 $39 $8 $64 $480 $188 $62 $129 $21 $138 $63 $31 $23 $24 $38 $85 $51 $19 $318 $361 $148 $36 $38 $32 $75 $60 $3 $26 $118 $337 $10 $105 $142 $73 $149 $109 $87 $7 $90 $42 $75 $71 $15 $33 $43 $50 $152 $37 $79 $167 $112 $193 $17 $12 $40 $4 $56 $21 $425 $36 $26 $192 $11 $224 $24 $46 $92 $110 $19 $49 $141 $11 $21 $126 $7 $37 $71 $43 $61 $104 $2 $148 $53 $255 $30 $141 $84 $42 $53 $134 $64 $56 $213 $108 $88 $59 $37 $34 $3 $54 $12 $163 $31 $71 $14 $78 $206 $12 $34 $121 $118 $32 $49 $19 $16 $159 $7 $27 $66 $24 $20 $43 $56 $59 $7 $62 $181 $14 $29 $48 $0 $150 $98 $104 $49 $140 $5 $205 $138 $74 $19 $319 $20 $21 $19 $269 $27 $36 $43 $56 $164 $15 $103 $14 $61 $26 $42 $13 $386 $10 $85 $50 $1 $10 $9 $60 $315 $64 $68 $81 $8 $89 $2 $120 $387 $13 $58 $28 $107 $39 $3 $67 $148 $27 $37 $50 $33 $36 $32 $59 $42 $28 $14 $33 

In [21]:
subset = 15_000
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=4)
rf_model.fit(X[:subset], prices[:subset])

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

## Random Forest model

In [22]:
def random_forest(item):
    x = vectorizer.transform([item.summary])
    return max(0, rf_model.predict(x)[0])

In [23]:
evaluate(random_forest, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$65 $60 $7 $15 $128 $150 $73 $55 $23 $241 $526 $295 $44 $80 $0 $9 $45 $98 $28 $53 $28 $25 $9 $31 $131 $305 $176 $106 $36 $47 $53 $82 $101 $20 $27 $324 $45 $20 $143 $83 $152 $52 $17 $49 $143 $31 $50 $39 $64 $8 $6 $27 $226 $36 $309 $40 $37 $147 $19 $55 $100 $3 $62 $5 $459 $5 $12 $189 $30 $120 $13 $24 $85 $134 $5 $99 $113 $24 $32 $74 $63 $78 $21 $49 $12 $29 $18 $110 $1 $98 $31 $174 $1 $80 $56 $83 $2 $42 $142 $283 $34 $5 $16 $73 $36 $23 $134 $200 $20 $184 $36 $73 $60 $34 $69 $50 $90 $11 $188 $50 $30 $41 $45 $34 $37 $25 $36 $61 $84 $6 $12 $52 $71 $21 $35 $63 $14 $10 $80 $33 $12 $156 $2 $97 $94 $67 $37 $330 $68 $2 $14 $104 $53 $73 $49 $86 $118 $40 $38 $10 $0 $1 $2 $31 $401 $45 $263 $31 $9 $35 $23 $5 $190 $27 $13 $59 $45 $9 $44 $8 $324 $19 $68 $89 $141 $56 $41 $94 $1 $41 $31 $61 $27 $1 $12 $24 $78 $32 $2 $13 

## XGBoost

In [25]:
import xgboost as xgb

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/devanshuprakash/Desktop/finetuned-llama-price-model/.venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <1A0D8152-BF46-3BE0-B651-EE965C187777> /Users/devanshuprakash/Desktop/finetuned-llama-price-model/.venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System

In [ ]:
np.random.seed(42)

xgb_model = xgb.XGBRegressor(n_estimators=1000, random_state=42, n_jobs=4, learning_rate=0.1)
xgb_model.fit(X, prices)

In [ ]:
def xg_boost(item):
    x = vectorizer.transform([item.summary])
    return max(0, xgb_model.predict(x)[0])

In [ ]:
evaluate(xg_boost, test)
